# Neural Network using Keras

### Importing necessary modules

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)

### Loading the data

In [2]:
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [3]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

### Define Model

Models in Keras are defined as a sequence of layers.

We create a Sequential model and add layers one at a time until we are happy with our network topology.

The first thing to get right is to ensure the input layer has the right number of inputs. This can be specified when creating the first layer with the input_dim argument and setting it to 8 for the 8 input variables.

How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation. Generally, you need a network large enough to capture the structure of the problem if that helps at all.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons in the layer as the first argument, the initialization method as the second argument as init and specify the activation function using the activation argument.

In this case, we initialize the network weights to a small random number generated from a uniform distribution (‘uniform‘), in this case between 0 and 0.05 because that is the default uniform weight initialization in Keras. Another traditional alternative would be ‘normal’ for small random numbers generated from a Gaussian distribution.

We will use the rectifier (‘relu‘) activation function on the first two layers and the sigmoid function in the output layer. It used to be the case that sigmoid and tanh activation functions were preferred for all layers. These days, better performance is achieved using the rectifier activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer. The first layer has 12 neurons and expects 8 input variables. The second hidden layer has 8 neurons and finally, the output layer has 1 neuron to predict the class (onset of diabetes or not).

In [4]:
# create model using Xavier initialisation and dropout on hidden layers
model1 = Sequential()
model1.add(Dense(12, input_dim=8,kernel_initializer='glorot_normal',activation='relu'))#using xavier initialisation
model1.add(Dropout(0.2))
model1.add(Dense(8, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))

In [5]:
# create model using Xavier initialisation and dropout on input layer
model2 = Sequential()
model2.add(Dropout(0.2, input_shape=(8,)))
model2.add(Dense(12,activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

### Compile Model

Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to make predictions for this problem.

We must specify the loss function to use to evaluate a set of weights, the optimizer used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as “binary_crossentropy“. We will also use the efficient gradient descent algorithm “adam” for no other reason that it is an efficient default. Learn more about the Adam optimization algorithm in the paper “Adam: A Method for Stochastic Optimization“.

Finally, because it is a classification problem, we will collect and report the classification accuracy as the metric.

In [6]:
# Compile model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit Model

We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the nepochs argument. We can also set the number of instances that are evaluated before a weight update in the network is performed, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of iterations (150) and use a relatively small batch size of 10. Again, these can be chosen experimentally by trial and error.

In [7]:
# Fit the model
model2.fit(X, Y, epochs=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 0s 403us/step - loss: 5.4267 - acc: 0.6016
Epoch 2/150
768/768 [==============================] - 0s 133us/step - loss: 5.1364 - acc: 0.6068
Epoch 3/150
768/768 [==============================] - 0s 129us/step - loss: 4.9104 - acc: 0.6211
Epoch 4/150
768/768 [==============================] - 0s 152us/step - loss: 4.7849 - acc: 0.6276
Epoch 5/150
768/768 [==============================] - 0s 157us/step - loss: 4.5114 - acc: 0.6354
Epoch 6/150
768/768 [==============================] - 0s 123us/step - loss: 4.3514 - acc: 0.6445
Epoch 7/150
768/768 [==============================] - 0s 155us/step - loss: 3.8090 - acc: 0.6302
Epoch 8/150
768/768 [==============================] - 0s 150us/step - loss: 3.1800 - acc: 0.6250
Epoch 9/150
768/768 [==============================] - 0s 151us/step - loss: 1.6474 - acc: 0.5456
Epoch 10/150
768/768 [==============================] - 0s 149us/step - loss: 1.0376 - acc: 0.5391
Epoch 11/150
768/76

### Evaluate Model

In [10]:
# evaluate the model
scores = model2.evaluate(X, Y)
print("\n%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

768/768 [==============================] - 0s 40us/step

acc: 75.78%


### Make Predictions

In [ ]:
# calculate predictions
predictions = model1.predict(X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded[:10])

### Saving a model along with its weights

In [11]:
# Save the weights
model2.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model2.to_json())

In [12]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

In [14]:
#Using the model
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

768/768 [==============================] - 0s 137us/step

acc: 75.78%


### saving and loading the entire model

In [16]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model2.save('my_model.h5')

# Deletes the existing model
#del model  

# Returns a compiled model identical to the previous one
model = load_model('my_model.h5')

In [18]:
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

768/768 [==============================] - 0s 197us/step

acc: 75.78%
